In [4]:
import pandas as pd
import sqlite3

# File paths - replace with your file paths
orders_csv = 'orders.csv'
line_items_csv = 'line_items.csv'
customers_csv = 'customers.csv'

# Create a connection to a new SQLite database
conn = sqlite3.connect('jiffyshirts.db')

# Import CSV files into the database
for file, table_name in [(orders_csv, 'orders'), (line_items_csv, 'line_items'), (customers_csv, 'customers')]:
    df = pd.read_csv(file)
    df.to_sql(table_name, conn, if_exists='replace', index=False)

# Close the connection
conn.close()


In [6]:
# Create a SQLite database
conn = sqlite3.connect('jiffyshirts.db')

# Check if the tables are created successfully
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
tables

,name
0,orders
1,line_items
2,customers


In [29]:
query_1 = """
SELECT COUNT(*) AS orders_in_2018
FROM orders
WHERE strftime('%Y', DATETIME(order_timestamp, '-5 hours')) = '2018'
"""

orders_2018 = pd.read_sql_query(query_1, conn)
orders_2018

,orders_in_2018
0,9228


In [35]:
# SQL Query 2: How many orders were completed in 2018 containing at least 10 units?

query_2 = """
SELECT COUNT(DISTINCT order_id) AS orders_with_10_units_2018
FROM (
    SELECT o.order_id, 
           SUM(li.quantity) AS total_quantity_per_order
    FROM orders o
    JOIN line_items li ON o.order_id = li.order_id
    WHERE strftime('%Y', DATETIME(o.order_timestamp, '-5 hours')) = '2018'
    GROUP BY 1
) AS subquery
WHERE total_quantity_per_order >= 10
"""

orders_with_10_units_2018 = pd.read_sql_query(query_2, conn)
orders_with_10_units_2018

,orders_with_10_units_2018
0,5151


In [13]:
# SQL Query 3: How many customers have ever purchased a medium-sized sweater with a discount?

query_3 = """
SELECT COUNT(DISTINCT o.customer_uid) AS customers_medium_sweater_discount
FROM orders o
JOIN line_items li ON o.order_id = li.order_id
WHERE li.size = 'M'
AND li.product_category = 'Sweater'
AND o.discount > 0
"""

customers_medium_sweater_discount = pd.read_sql_query(query_3, conn)
customers_medium_sweater_discount

,customers_medium_sweater_discount
0,753


In [15]:
# SQL Query 4: What was the profitability of our most profitable month?

query_4 = """
SELECT strftime('%Y-%m', DATETIME(o.order_timestamp, '-5 hours')) AS month,
       SUM((li.selling_price - li.supplier_cost) * li.quantity) AS total_profit
FROM orders o
JOIN line_items li ON o.order_id = li.order_id
GROUP BY month
ORDER BY total_profit DESC
LIMIT 1
"""

most_profitable_month = pd.read_sql_query(query_4, conn)
most_profitable_month

,month,total_profit
0,2020-10,92431.26


In [22]:
# SQL Query 5: What is the return rate for business vs. non-business customers?

query_5 = """
SELECT c.is_business,
       COUNT(*) AS total_orders,
       SUM(CASE WHEN o.returned = True THEN 1 ELSE 0 END) AS returned_orders,
       (SUM(CASE WHEN o.returned = True THEN 1 ELSE 0 END) * 1.0 / COUNT(*))*100 AS return_rate_percent
FROM customers c
JOIN orders o ON c.customer_uid = o.customer_uid
GROUP BY c.is_business
"""

return_rate_business_nonbusiness = pd.read_sql_query(query_5, conn)
return_rate_business_nonbusiness

,is_business,total_orders,returned_orders,return_rate_percent
0,0,18278,900,4.923952
1,1,28155,1899,6.744806
